# Importation and Initialisation

In [12]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from firebase import FirebaseManager 
from mongo import MongoDBManager
from selenium.webdriver.common.keys import Keys
import os
from dotenv import load_dotenv
from tqdm.notebook import tqdm

In [2]:
collection_names=["professions_by_field","posts_by_profession"]
# credential_path="./indeedrecommender-firebase.json"
# firebase_manager=FirebaseManager(credential_path)

In [3]:
load_dotenv()
uri = os.getenv('MONGODB_URI')
database_name = os.getenv('DATABASE_NAME')
mongo_manager = MongoDBManager(uri, database_name)

# Retrieve Fields and theirs professions from Ulaval Website

In [3]:
def retrieve_professions_from_field_first(field):
    """
    Retrieves professions from the field using the provided field parameter.

    Args:
        field (str): The field to retrieve professions from.

    Raises:
        Exception: If no elements are found for the first function for the given field.

    Returns:
        None
    """
    global datas
    h3_elements = driver.find_elements(By.XPATH, "//h3[contains(text(), 'Professions')]")
    print(h3_elements)
    if len(h3_elements)==0:
        raise Exception(f"Aucun element trouvé pour la première fonction pour {field}")
    for h3_element in h3_elements:
        ul_element = h3_element.find_element(By.XPATH, "following-sibling::ul")
        driver.execute_script("arguments[0].scrollIntoView(true);", h3_element)
        if ul_element:
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                # print(text)
                if text not in datas[field]:
                    datas[field].append(text)
            # print(datas)
        time.sleep(0.5)
    # driver.back()



def retrieve_professions_from_field_second(field):
    """
    Retrieves professions from the second field.

    Args:
        field (str): The field to retrieve professions from.

    Raises:
        Exception: If no elements are found for the second function for the given field.

    Returns:
        None
    """
    global datas
    strong_elements = driver.find_elements(By.XPATH, "//p[strong[contains(text(), 'Professions')]]")
    print(f"strong: {strong_elements}")
    if len(strong_elements)==0:
        errors.append(field)
        raise Exception(f"Aucun element trouvé pour la deuxième fonction pour {field}")
    for strong_element in strong_elements:
        ul_element = strong_element.find_element(By.XPATH, "following-sibling::ul")
        if ul_element:
            driver.execute_script("arguments[0].scrollIntoView(true);", ul_element)
            li_elements = ul_element.find_elements(By.XPATH, "./li")
            for element in li_elements:
                text = driver.execute_script("return arguments[0].innerText;", element)
                # ##################print(text)
                if text not in datas[field]:
                    datas[field].append(text)
            # print(datas)
        time.sleep(0.5)
    # driver.back()

def retrieve_professions_from_field(field):
    global nb_of_functions_tried, nb_of_functions
    try:
        if nb_of_functions_tried==1:
            retrieve_professions_from_field_first(field)
            driver.back()
        if nb_of_functions_tried==2:
            retrieve_professions_from_field_second(field)
            driver.back()
            nb_of_functions_tried=1
    except Exception as e:
        print(e)
        nb_of_functions_tried+=1
        retrieve_professions_from_field(field)

    if nb_of_functions_tried > nb_of_functions:
        driver.back()
        nb_of_functions_tried=1
    return



In [ ]:
driver = webdriver.Chrome()
driver.get("https://www.ulaval.ca/etudes/programmes")
title = driver.title

# Keys represente fields and values are arrays whose contains professions 
datas={}
errors=[]
nb_of_functions=2
nb_of_functions_tried=1

pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
print(pagination_next_element)
while pagination_next_element:
    results_element= driver.find_element(By.ID,"resultats")
    children=results_element.find_elements(By.XPATH, "./ul[@class='views-row']")

    for child in children:
        element=child.find_element(By.XPATH, "./li/a")
        field=element.find_element(By.XPATH,"./div[@class='main-info']/h3[@class='titre-programme']").text
        if field not in datas:
            datas[field]=[]     
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        time.sleep(0.5)
        driver.execute_script("arguments[0].click();", element)
        retrieve_professions_from_field(field)
    
    driver.execute_script("arguments[0].scrollIntoView(true);", pagination_next_element)
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", pagination_next_element.find_element(By.TAG_NAME, "a"))
    try:
        pagination_next_element=driver.find_element(By.ID,"pagination-heading").find_element(By.XPATH, "following-sibling::ul").find_element(By.XPATH, "./li[@class='pagination-item pagination-item--suivant']")
    except Exception as e:
        pagination_next_element=None
   
driver.close()
print(datas)


In [5]:
for key in datas.keys():
    mongo_manager.add_data_to_mongodb(collection_names[0],key,{"professions":datas[key]})


Data added to professions_by_field/actuariat
Data added to professions_by_field/actuariat - avec mémoire
Data added to professions_by_field/administration des affaires
Data added to professions_by_field/administration des affaires - Global Business
Data added to professions_by_field/administration des affaires - analytique d'affaires
Data added to professions_by_field/administration des affaires - audit interne, contrôle et gestion des risques
Data added to professions_by_field/administration des affaires - chaînes de blocs pour gestionnaires
Data added to professions_by_field/administration des affaires - communication marketing sur les nouveaux médias
Data added to professions_by_field/administration des affaires - comptabilité
Data added to professions_by_field/administration des affaires - comptabilité et décisions d'affaires
Data added to professions_by_field/administration des affaires - comptabilité pour gestionnaires
Data added to professions_by_field/administration des affaire

# Retrieve Posts from Indeed

In [4]:
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

def stateDateToDate(stateDate):
   """
   Converts a state date string into a datetime object.
   Parameters:
   stateDate (str): The state date string to be converted.
   Returns:
   datetime: The converted datetime object.
   """

   date=None
   value=None
   for str in stateDate.split(" "):
      try:
         date= datetime.strptime(str, '%Y-%m-%d')
      except Exception as e:
         date=None
      
      if date:
         return date
   try:
      match = re.search(r'\d+', stateDate)
      if match:
         value= int(match.group())
   except Exception as e:
      e

   if value:
      if bool(re.compile(r'\b(\w*jour\w*|\w*day\w*)\b', re.IGNORECASE).search(stateDate)):
         date= (datetime.now()-relativedelta(days=value)).strftime('%Y-%m-%d')
      if bool(re.compile(r'\b(\w*mois\w*|\w*month\w*)\b', re.IGNORECASE).search(stateDate)):
         date= (datetime.now()-relativedelta(months=value)).strftime('%Y-%m-%d')
      if bool(re.compile(r'\b(\w*an \w*|\w*year\w*|\w*année\w*)\b', re.IGNORECASE).search(stateDate)):
         date= (datetime.now()-relativedelta(years=value)).strftime('%Y-%m-%d')
      if bool(re.compile(r'\b(\w*now\w*|\w*maintenant\w*|\w*instant\w*)\b', re.IGNORECASE).search(stateDate)):
         date= (datetime.now()).strftime('%Y-%m-%d')            
   return date

In [5]:
positionsDetailsList = {}

def indeedPositionInfos(profession, country):
    """
    Retrieves position details from Indeed job listings.
    Args:
        profession (str): The profession to search for.
    Returns:
        None
    Raises:
        None
    """
    global positionsDetailsList
    positionsDetails={}
    companyName=''
    location=''
    jobType=''
    jobBenefits=[]
    benefits=''
    shiftAndSchedule=''
    description=''
    positionPostedDate=''
    postedText=''
    salary=''
    try:
        positionsDescriptionList = driver.find_element(By.XPATH, "//div[@id='mosaic-provider-jobcards']")
        ulElement = positionsDescriptionList.find_element(By.XPATH, "./ul")
        liElements = ulElement.find_elements(By.XPATH, "./li[@class='css-5lfssm eu4oa1w0']")
    except Exception as e:
        liElements=[]
    for li in liElements:
        positionsDetails = {}
        driver.execute_script("arguments[0].scrollIntoView(true);", li)
        time.sleep(0.5)
        try:
          position=li.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]/a")
          positionTitle = driver.execute_script("return arguments[0].innerText;", position)
          print(positionTitle)
        except Exception as e:
            print("No position found")
            position=None

        try:
          dateElement=li.find_element(By.XPATH, ".//span[@data-testid='myJobsStateDate']")
          postedText=driver.execute_script("return arguments[0].innerText.replace('Posted', '').trim().replace(/\\n/g, ' ');", dateElement)
          positionPostedDate=stateDateToDate(postedText)
        except Exception as e:
            print("No Date found")
            dateElement=None
        
        if position :
            driver.execute_script("arguments[0].click();", position)
            time.sleep(0.5)
            try:
               positionsDetailsElement = driver.find_element(By.XPATH, "//div[@id='jobsearch-ViewjobPaneWrapper']")
            except Exception as e:
               positionsDetailsElement=None
               print("No position details found")
               
            
            if positionsDetailsElement:	
                try:
                    companyName = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='inlineHeader-companyName']"))
                except Exception as e:
                    print("No company name found")
                    companyName=''
                try:
                    location = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='inlineHeader-companyLocation']"))
                except Exception as e:  
                    print("No location found")
                    location=''
                try:
                    jobType = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.ID, "salaryInfoAndJobType"))
                    print(F"JOB type: {jobType}")
                except Exception as e:
                    print("No job type found")
                    jobType=''
                try:
                    jobType2 = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.ID,"jobDetailsSection").find_element(By.XPATH, ".//div[@data-testid='Permanent-tile']"))
                    print(F"JOB type 2: {jobType2}")
                except Exception as e:
                    print("No job type 2 found")
                    jobType2=""
                try:
                    jobType3 = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='Full-time-tile']"))
                    print(F"JOB type 3: {jobType3}")
                except Exception as e:
                    print("No job type 3 found")
                    jobType3=""
                try:
                    jobType4 = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@data-testid='Part-time-tile']"))
                    print(F"JOB type 4: {jobType4}")
                except Exception as e:
                    print("No job type 4 found")
                    jobType4=""
                
                if re.search(r'-', jobType):
                    salary = jobType.split("-")[0]
                    jobType= jobType.split("-")[1]
                print(F"Salary: {salary}")
                jobType=jobType.replace(salary,'')
                print(F"JOB type: {jobType}")
                jobType=jobType.split(",")
                if jobType2 not in jobType:
                    jobType.append(jobType2)
                if jobType3 not in jobType:
                    jobType.append(jobType3)
                if jobType4 not in jobType:
                    jobType.append(jobType4)
                jobType.remove('')
                try:
                    shiftAndSchedule = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH,".//h3[contains(text(), 'Quart de travail et horaire')]").find_element(By.XPATH, "following-sibling::div"))
                    print(F"Shift and schedule: {shiftAndSchedule}")
                except Exception as e:
                    print("No shift and schedule found")
                    shiftAndSchedule=''
                jobBenefits = []
                try:
                    benefits = positionsDetailsElement.find_elements(By.XPATH, ".//div[@id='benefits']/div/div/span/ul[@class='css-8tnble eu4oa1w0']/li")
                    for benefit in benefits:
                        jobBenefits.append(driver.execute_script("return arguments[0].innerText;",benefit))
                
                except Exception as e:
                    # print("No benefits found")
                    benefits=''
                    
                try:
                    description = driver.execute_script("return arguments[0].innerText;",positionsDetailsElement.find_element(By.XPATH, ".//div[@id='jobDescriptionText']"))
                except Exception as e:
                    print("No description found")
                    description=''
        
        positionsDetails["positionTitle"]=positionTitle
        positionsDetails["companyName"]=companyName
        positionsDetails["location"]=location
        positionsDetails["jobType"]=jobType
        positionsDetails["description"]=description
        positionsDetails["datePosted"]=positionPostedDate
        positionsDetails["salary"]=salary
        positionsDetails["shiftAndSchedule"]=shiftAndSchedule
        positionsDetails["jobBenefits"]=jobBenefits
        if country not in positionsDetailsList:
            positionsDetailsList[country]={}
        if profession not in positionsDetailsList[country]:
            positionsDetailsList[country]={profession:[]}
        positionsDetailsList[country][profession].append(positionsDetails)
        time.sleep(2)


In [6]:
professions_by_field=mongo_manager.read_all_documents_in_collection(collection_names[0])
print(professions_by_field)
professions=[]
for key in professions_by_field.keys():
    professions.extend(professions_by_field[key]["professions"])
print(professions)

{'actuariat': {'_id': 'actuariat', 'professions': ['Actuaire', 'Analyste de marché', 'Analyste en assurances', 'Analyste en placements', 'Analyste en finance']}, 'actuariat - avec mémoire': {'_id': 'actuariat - avec mémoire', 'professions': []}, 'administration des affaires': {'_id': 'administration des affaires', 'professions': ["Agente ou agent d'administration", "Chargée ou chargé de développement d'affaires", 'Directrice administrative ou directeur administratif', 'Gestionnaire', 'Chargée ou chargé de projet', 'Cheffe ou chef de service ou de division', 'Consultante ou consultant', 'Gestionnaire/décideuse ou décideur', 'Experte ou expert en science des données (data scientist)', "Analyste en gestion de l'actif", 'Analyste financière ou analyste financier', 'Conseillère ou conseiller en gestion budgétaire et financière', 'Conseillère ou conseiller en placement', 'Courtière ou courtier en valeurs mobilières', 'Gestionnaire de portefeuille', 'Gestionnaire de risques (CRM)', 'Négociatr

In [7]:
countries=["Canada","US","France"]
languages=["fr","en"]

In [8]:
import random
from selenium.webdriver.chrome.options import Options


USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
]

# def get_random_proxy():
#     return random.choice(PROXIES)


chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument(f'user-agent={random.choice(USER_AGENTS)}')


def random_sleep(min_time=1, max_time=5):
    time.sleep(random.uniform(min_time, max_time))

def indeedDataFill(ulavalPosition, location):
    if location=="Canada":
        driver.get(f"https://emplois.ca.indeed.com/jobs?q={ulavalPosition.replace(' ','+')}&l={location.replace(' ','+')}")
    if location=="US":
        driver.get(f"https://www.indeed.com/jobs?q={ulavalPosition.replace(' ','+')}&l={location.replace(' ','+')}")
    if location=="France":
        driver.get(f"https://www.indeed.fr/jobs?q={ulavalPosition.replace(' ','+')}&l={location .replace(' ','+')}")    
    random_sleep()
    # try:
    #     title_input = driver.find_element(By.ID, 'text-input-what')
    #     location_input = driver.find_element(By.ID, 'text-input-where')
    # except Exception as e:
    #     e
    # if title_input:
    #     driver.execute_script("arguments[0].value = '';", title_input)
    #     # driver.execute_script("arguments[0].value = '';", location_input)
    #     # location_input.send_keys(location)
    #     title_input.send_keys(ulavalPosition, Keys.RETURN)
    # return title_input, location_input
    

In [9]:
import requests

def translate_text(text, target_language):
    url = "https://api.mymemory.translated.net/get"
    params = {
        'q': text,
        'langpair': f'fr|{target_language}'  # Exemple pour traduire du français vers l'anglais
    }
    response = requests.get(url, params=params)
    return response.json().get('responseData', {}).get('translatedText')

def traduction(profession, language):
    if language=="fr":
        return profession
    if language=="en":
        return translate_text(profession, "en")
    
traduction("Infirmier", "en")

'Nurse'

In [14]:
try:
    driver = webdriver.Chrome(options=chrome_options)
    for profession in tqdm(professions):
        for country in tqdm(countries):
            for language in tqdm(languages):
                indeedDataFill(traduction(profession,language), country)
                try:
                    pagination_next_element=driver.find_element(By.XPATH, "//nav/ul/li/a[@data-testid='pagination-page-next']")
                except Exception as e:
                    pagination_next_element=None
                    


                if not pagination_next_element: 
                    print(f"Boutton {pagination_next_element}")
                    indeedPositionInfos(profession, country)
                if pagination_next_element:
                    while pagination_next_element:
                        indeedPositionInfos(profession, country)
                        if pagination_next_element:
                            driver.execute_script("arguments[0].click();", pagination_next_element)
                        try:
                            pagination_next_element=driver.find_element(By.XPATH, "//nav/ul/li/a[@data-testid='pagination-page-next']")
                        except Exception as e:
                            pagination_next_element=None
    driver.close()
except Exception as e:
    print(e)
    driver.close()

  0%|          | 0/2244 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Administrateur de régime de retraite
JOB type: Permanent, Temps plein
JOB type 2: Permanent
No job type 3 found
No job type 4 found
Salary: 
JOB type: Permanent, Temps plein
Shift and schedule: Du Lundi au Vendredi
Actuaire principal ou actuaire principale
JOB type: Permanent
JOB type 2: Permanent
No job type 3 found
No job type 4 found
Salary: 
JOB type: Permanent
Shift and schedule: Heures supplémentaires
Directeur(trice), relations avec la clientèle, Solutions prestations déterminées
JOB type: 110 000 $ par an
No job type 2 found
No job type 3 found
No job type 4 found
Salary: 
JOB type: 110 000 $ par an
No shift and schedule found
Opportunités de carrière en actuariat
JOB type: Permanent
JOB type 2: Permanent
No job type 3 found
No job type 4 found
Salary: 
JOB type: Permanent
No shift and schedule found
Directeur(rice) général(e), Programmes de pension et de retraite
No job type found
No job type 2 found
No job type 3 found
No job type 4 found
Salary: 
JOB type: 
No shift and sche

In [88]:
((positionsDetailsList["Canada"]["Actuaire"][:10]))

[{'positionTitle': 'Administrateur de régime de retraite',
  'companyName': 'Gamma Actuaires Conseils Inc',
  'location': '1420 Rue Sherbrooke O, Montréal, QC',
  'jobType': ['Permanent', ' Temps plein'],
  'description': "Présentation de l'employeur\n\nGamma Actuaires Conseils Inc. offre des services de consultation et d’actuariat pour les régimes de retraite.\n\nDescription du poste\n\nNous cherchons à combler un poste permanent à temps plein en administration des régimes de retraite au sein de notre équipe. Le nouvel employé fera partie d’une équipe dynamique chargé de la mise en place, de l’administration et des évaluations actuarielles pour les régimes de retraite individuels (RRI). Le candidat sera appelé à:\n\n- Préparer des projections actuarielles pour les clients potentiels;\n\n- Produire les documents requis pour l’enregistrement de nouveaux RRI;\n\n- Préparer les évaluations actuarielles;\n\n- Préparer les documents nécessaires pour faire des amendements aux RRI\n\n- Prépar

In [ ]:
for key in tqdm(positionsDetailsList.keys()):
    mongo_manager.add_data_to_mongodb(collection_names[1],key,{"positions":positionsDetailsList[key]})

Error adding data to Firestore: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNAVAILABLE: ipv4:172.217.13.138:443: No route to host
Error adding data to Firestore: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNAVAILABLE: ipv4:172.217.13.138:443: No route to host
Error adding data to Firestore: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNAVAILABLE: ipv4:172.217.13.138:443: No route to host
Error adding data to Firestore: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNAVAILABLE: ipv4:172.217.13.138:443: No route to host
Error adding data to Firestore: Timeout of 60.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNAVAILABLE: ipv4:172.217.13.138:443: No route to host
Error adding data to Firestore: Timeout of 60.0s exceeded, last exception: 503 failed to c